### Set Up

In [ ]:
import findspark 
findspark.init() 

import pyspark
from delta import *
import pandas as pd
from pyspark.sql.functions import col

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
spark = pyspark.sql.SparkSession.builder.master("spark://spark-master:7077").appName("bronze_silver").getOrCreate()

sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [ ]:
import sys
sys.path.append('/opt/spark/work-dir/odp_intra_storage/vysh_temp/ingestion_framework/')

### Silver Metadata

In [ ]:
spark.sql("create database if not exists silver").toPandas()

In [ ]:
spark.sql("show databases").toPandas()

In [ ]:
from metadata_refresh.silver_refresh import refresh_silver_metadata

In [ ]:
env = 'test'
silver_metadata_file_location = '/opt/spark/work-dir/odp_intra_storage/vysh_temp/kubernetes/spark_clients/notebooks/silver/silver_metadata.csv'
silver_metadata_table_name = 'silver.metadata_table'

spark.sql(f"drop table if exists {silver_metadata_table_name}")

refresh_silver_metadata(spark, env, silver_metadata_file_location, silver_metadata_table_name)

In [ ]:
spark.table("silver.metadata_table").where("pipeline_id = 1").select("*").toPandas()

### Streaming Pipeline

In [ ]:
from spark_ingestion_app.spark_ingestion_app import SparkIngestionApp

In [ ]:
# spark.sql("drop table if exists silver.weather_v1_events")

In [ ]:
spark.sql("""
create table if not exists silver.weather_v1_events
(
message_key string,
load_time timestamp,
kafka_topic string,
kafka_partition int,
kafka_offset bigint,
kafka_timestamp timestamp,
kafka_date date,
message_sent_at timestamp,
location string,
time string,
info string,
temperature string,
precipitation string,
humidity string,
wind string
)
using delta
""")

In [ ]:
module_parameters = '\'{"group_id": "1", "pipeline_id": "1"}\''

SparkIngestionApp.run_module(spark = spark, module_name = 'BronzeToSilver', module_parameters = module_parameters, log_level='INFO')

In [ ]:
# spark = pyspark.sql.SparkSession.builder.master("local").getOrCreate()
spark.sql("select * from silver.weather_v1_events order by kafka_timestamp desc limit 10").toPandas()